# PARSING новостей - Набор СКРИПТОВ для работы в автономном режиме (сохранённые HTML коды)

In [ ]:
# Подключаем библиотеку для поиска элементов кода на сайте !!!!!!!!!!!!!
# Библиотека PYTHON для извлечения данных из файлов HTML и XML
from bs4 import BeautifulSoup

# Подключаем библиотеки для парсинга сайтов !!!!!!!!!!!!!
import requests
import lxml
import os
import json
import csv

# Работа с файлами XLSX
import openpyxl
import xlsxwriter

# Использование объекта datetime :
from datetime import datetime
from time import sleep
import time
import random

In [2]:
# Выполним импорты for GoogleSheets:
# from gspread import Client, Spreadsheet, Worksheet, service_account
from gspread import Client, Spreadsheet, Worksheet, service_account, exceptions
from typing import List, Dict

In [ ]:
# Модули для парсинга динамических сайтов
from selenium import webdriver

from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

## Константы для проекта

# ФУНКЦИЯ № 1. Определение переменных

In [6]:
def variable():
# 0. Создаём словарь заголовков
    headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
        }
    
# 1. Список адресов основного хоста
    hosts = ['https://vc.ru']
    
# 2. Список адресов страниц для парсинга
    urls = ['https://vc.ru/new']

# 3. Список имен сайтов - список словарей
    names = [{"name": u.split('//')[1].split('.')[0]} for u in urls]
    
# 4. Словарь путей для папок с данными
    type_files = ['csv', 'html', 'jpg', 'json', 'txt', 'xlsx']
    path_files = ['../data/processed/csv/', '../data/interim/html/', '../data/processed/img/',
                 '../data/processed/json/', '../data/processed/txt/', '../data/processed/xlsx/'
                ]
    extension_files = ['.csv', '.html', '.jpg', '.json', '.txt', '.xlsx']
    
    paths = [{'type_file': type_files[i], 'path_file': path_files[i], 'extension_file': extension_files[i]} for i in range(len(type_files))]
    
# 5. Словарь атрибутов Супа - список списков
    divs = ['div', 'p']
    classes = ['desc list', 'news-item']
    pattern = [[divs[i], classes[i]] for i in range(len(divs))]
    
    
    return headers, hosts, urls, names, paths, pattern

# ФУНКЦИЯ № 2. Сохранение HTML-кода для тестирования

In [7]:
# import os
# Отправляем запрос на страницу, передаём в качестве GET-параметра URL-адрес
# Сохраним нашу страницу в файл И ЗАКОММЕНТИРУЕМ ЭТУ КОМАНДУ за ненадобностью

def write_html(headers, url, path):
    # path = '../data/interim/html/'
# Если папки для сохранения результатов не существует, то она будет создана
    if not os.path.isdir(path[1]['path_file']):
        print(f'Fun.2.0. Folder "{path[1]['path_file']}" does NOT exist - it will be created !!!')
        os.mkdir(path[1]['path_file'])
    else:
        print(f'Fun.2.0. Folder "{path[1]['path_file']}" FOUND!!! It`s OK')
    
    print(f'Fun.2.0. START of function "WRITE_HTML()"\n')
    
    count = 1
# Проходим циклом по списку со ссылками на страницы сайтов
    for u in url:
        r = requests.get(url=u, headers=headers)
        src = r.text

        file_start = 'news_articles_'
        name = u.split('//')[1].split('.')[0]
        file_name = file_start + name
        file_path = os.path.join(path[1]['path_file'], file_name) + path[1]['extension_file']
        print(f'{count}. Function "WRITE_HTML({name})" EXECUTING ...')

# Проверка существования старых файлов с таким же названием (если они существуют - обновляем)
        if not os.path.exists(file_path):
            print(f"\tThere is NO file \"{file_name}\" in the folder \"{path[1]['path_file']}\" - It will be CREATED.")
        else:
            print(f"\tIn the folder \"{path[1]['path_file']}\" the file \"{file_name}\" EXISTS.")
            print(f'\tThe file "{file_name}" will be DELETED and REWRITTEN.')
            os.remove(file_path)
            
# Сохраняем код по указанному пути
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(src)
        print(f'\tHTML-code \"{file_name}\" is SAVED in folder \"{path[1]['path_file']}\"')
        print(f'Function "WRITE_HTML({name.upper()})" is EXECUTED\n')
        count += 1

    print(f'Fun.2.0. Function "WRITE_HTML()" COMPLETED successfully!')

# Безусловный запуск функции
if __name__ == '__main__':
    write_html(variable()[0], variable()[2], variable()[4])

Fun.2.0. Folder "../data/interim/html/" FOUND!!! It`s OK
Fun.2.0. START of function "WRITE_HTML()"

1. Function "WRITE_HTML(vc)" EXECUTING ...
	In the folder "../data/interim/html/" the file "news_articles_vc" EXISTS.
	The file "news_articles_vc" will be DELETED and REWRITTEN.
	HTML-code "news_articles_vc" is SAVED in folder "../data/interim/html/"
Function "WRITE_HTML(VC)" is EXECUTED

Fun.2.0. Function "WRITE_HTML()" COMPLETED successfully!


### СОХРАНИЛИ ФАЙЛЫ В НУЖНОЙ ПАПКЕ !!!!

# ФУНКЦИЯ № 3.0. Чтение HTML-кода, сохранение его в список

In [4]:
# READ_HTML:
def read_html(name, path):
    print(f'Fun.3.0. START of function "READ_HTML()"')
    count = 1
    
    # print(name)
    file_start = 'news_articles_'
    file_name = file_start + name
    file_path = os.path.join(path['path_file'], file_name) + path['extension_file']
    print(f'{count}. Function "READ_HTML({name.upper()})" is EXECUTING...')
    
# Проверка существования файлов с HTML-кодом. Существующие коды собираем в список.
    if os.path.exists(file_path):
        print(f"\tIn the folder \"{path['path_file']}\" the file \"{file_name}\" EXISTS.")
        
        with open(file_path, encoding='utf-8') as file:
            src = file.read()
            
        print(f'\tHTML-code "{file_name}" is OPENED from {file_path}')
    else:
        print(f"\tThere is NO file \"{file_name}\" in the \"{path['path_file']}\" folder!")
        print('\tIt is necessary to START function "WRITE_HTML()" !!!\n')
        # break
        
    count += 1
    print(f'Fun.3.0. "READ_HTML({name.upper()})" COMPLETED SUCCESSFULLY!')

    return src
    # yield src

# ПОТОМ НУЖНО СДЕЛАТЬ ВМЕСТО RETURN --- YIELD !!!!!!

In [5]:
read_html(variable()[3][0]['name'], variable()[4][1])

Fun.3.0. START of function "READ_HTML()"
1. Function "READ_HTML(VC)" is EXECUTING...
	In the folder "../data/interim/html/" the file "news_articles_vc" EXISTS.
	HTML-code "news_articles_vc" is OPENED from ../data/interim/html/news_articles_vc.html
Fun.3.0. "READ_HTML(VC)" COMPLETED SUCCESSFULLY!


'<!DOCTYPE html>\n<html  data-theme="auto"  lang="en">\n  <head>\n    <meta charset="UTF-8" />\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, viewport-fit=cover, user-scalable=no" />\n    <meta name="apple-mobile-web-app-capable" content="yes">\n    <title>vc.ru — бизнес, технологии, идеи, модели роста, стартапы</title>\n<meta property="og:site_name" content="vc.ru">\n<meta property="og:image:width" content="600">\n<meta property="og:image:height" content="315">\n<meta name="twitter:card" content="summary_large_image">\n<meta property="og:url" content="https://vc.ru/popular">\n<meta name="description" content="Крупнейшая в рунете площадка для предпринимателей нового поколения. Новости ИТ-рынка, вакансии рунета, реклама, PR, интерфейсы, мобильные приложения.">\n<meta property="og:type" content="website">\n<meta property="og:image" content="https://api.vc.ru/v2.1/cover/fb/general/cover.jpg">\n<meta name="twitter:image" content="https://api.vc.ru/v2.1/cover/tw/

# ФУНКЦИЯ № 5.0. Проверка наличия папки для файлов любого типа (CSV, TXT, XLSX...)

In [72]:
def checking_folder(path):
    # path = '../data/processed/csv/' = variable()[4][0]['path_file']
    
# Если папки для сохранения результатов не существует, то она будет создана
    if not os.path.isdir(path['path_file']):
        try:
            print(f"Fun.5.0. Folder \"{path['type_file'].upper()}\" does NOT exist - it will be CREATED !!!\n")
            os.mkdir(path['path_file'])
        except KeyError as e:
            print(f"Fun.5.0. Sorry, but you have the folder name WRONG! There CAN't BE a folder \"{path}\"! % str(e)")
    else:
        print(f"Fun.5.0. Folder \"{path['type_file'].upper()}\" FOUND!!! It`s OK\n")

    # return path

In [73]:
# checking_folder(variable()[4][0])
# checking_folder('../data/processed/csv/')

checking_folder(variable()[4][5])
# checking_folder('../data/processed/xlsx/')

Fun.5.0. Folder "XLSX" FOUND!!! It`s OK



# ФУНКЦИЯ № 6.0. Проверка наличия старых CSV файлов в папке, создание заголовков

In [70]:
# ЕСЛИ ФАЙЛ УЖЕ ИМЕЕТСЯ, ТО НИЧЕГО ДЕЛАТЬ НЕ НУЖНО - PREP_CSV(path, name)
# Для подготовки CSV файла в YIELD функции - def csv_preparation(path, name)
def prep_csv(name, path):
    
    print(f'\tFun.6.0. START "PREP_CSV({name.upper()})"')
    
    file_start = 'news_articles_'
    file_name = file_start + name
    file_path = os.path.join(path['path_file'], file_name) + path['extension_file']
        
# Проверка существования старых CSV файлов с таким же названием (если они существуют - НИЧЕГО С НИМИ НЕ ДЕЛАЕМ).
    if os.path.exists(file_path):
        print(f"\t\tThe file \"{file_name}\" already EXISTS in folder \"{path['type_file'].upper()}\".")
    else:
        print(f"\t\tThere is NO file \"{file_name}\" in the folder \"{path['path_file'].upper()}\" - It will be CREATED.\n")
                
        # Собираем заголовки для CSV-файла
        with open(file_path, 'w', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(
                (
                    'Number',
                    'Author',
                    'Date',
                    'Topic',
                    'Shorts',
                    'Link',
                    'DateTime'
                )
            )
    
    print(f'\tFun.6.0. END "PREP_CSV({name.upper()})"')

In [71]:
prep_csv('chelyabinsk', variable()[4][0])

	Fun.6.1. START "PREP_CSV(CHELYABINSK)"
		The file "barrels_articles_chelyabinsk" already EXISTS in folder "CSV".
	Fun.6.1. END "PREP_CSV(CHELYABINSK)"


# ФУНКЦИЯ № 7.0. Создание CSV-файла

In [74]:
# Для заполнения CSV файла содержимым построчно - def save_csv(name, count, title, link, date_time, path):
def save_csv(name, count, title, link, date_time, path):
    
    file_start = 'news_articles_'
    file_name = file_start + name
    file_path = os.path.join(path['path_file'], file_name) + path['extension_file']
    
# Заполняем CSV файл построчно (внутри цикла); "А" - APPEND - добавление записей 
    with open(file_path, 'a', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(
            (
                count,
                title,
                link,
                date_time
            )
        )

In [87]:
# save_csv(urls)

# ФУНКЦИЯ № 9. Проверка наличия папки для JSON-файлов, старых JSON-файлов и создание нового JSON-файла

In [75]:
def save_json(name, path, articles_list):

# Если папки для сохранения результатов не существует, то она будет создана
    if not os.path.isdir(path['path_file']):
        print(f"\nFun.9.0. Folder \"{path['type_file'].upper()}\" does NOT exist - it will be CREATED !!!")
        os.mkdir(path['path_file'])
    else:
        print(f"\nFun.9.0 Folder \"{path['type_file'].upper()}\" FOUND!!! It`s OK")


    file_start = 'news_articles_'
    file_name = file_start + name
    file_path = os.path.join(path['path_file'], file_name) + path['extension_file']
    print(f'Fun.9.0. START function SAVE_JSON("{name.upper()}")')
    
# Проверка существования старых JSON файлов с таким же названием (если они существуют - обновляем).
    if not os.path.exists(file_path):
        print(f"\tThere is NO file \"{file_name}\" in the folder \"{path['path_file'].upper()}\" - It will be CREATED.")
    else:
        print(f"\tIn the folder \"{path['type_file'].upper()}\" the file \"{file_name}\" EXISTS.")
        print(f'\tThe file "{file_name}" will be DELETED and REWRITTEN.')
        os.remove(file_path) # DON'T FORGET TO OPEN THE LINE !!!!!!!!!!!!!!!!!!!!!!!!!!!!

    
# Создаём третий файл в формате JSON, в который будем записывать данные в виде словаря
# который весьма полезен и удобен для чтения (в дополнение к формату CSV)
    with open(file_path, 'a', encoding='utf-8') as file:
        json.dump(articles_list, file, indent=4, ensure_ascii=False)

    print(f"Fun.9.0. FINISH function SAVE_JSON(\"{name.upper()}\")")

# ФУНКЦИЯ № 10. Чтение HTML-кода, БЕЗ сохранения его в список. Команда YIELD.

In [76]:
# Функция для чтения сохранённых HTML-файлов в HTML папке - выбор по имени
def read_html_yield_names(name, path, count):
    print(f'{count+76}. Fun.10.0. START of function "READ_HTML_YIELD_NAMES()"')
    
    file_start = 'news_articles_'
    file_name = file_start + name
    file_path = os.path.join(path['path_file'], file_name) + path['extension_file']
    print(f"\tFunction \"READ_HTML_YIELD_NAMES({name.upper()})\" is EXECUTING...")
    
# Проверка существования файлов с HTML-кодом. Существующие коды собираем в список.
    if os.path.exists(file_path):
        print(f"\tIn the folder \"{path['type_file'].upper()}\" the file \"{file_name}\" EXISTS.")
        
        with open(file_path, encoding='utf-8') as file:
            src = file.read()
            
        print(f"\t\"{path['type_file'].upper()}\"-code \"{file_name}\" is OPENED from \"{file_path}\"")
    else:
        print(f"\tThere is NO file \"{file_name}\" in the \"{path['type_file'].upper()}\" folder!")
        print('\tIt is necessary to START function "WRITE_HTML()" !!!\n')
        # break
        
    print(f'{count+76}. Fun.10.0. "READ_HTML_YIELD_NAMES({name.upper()})" COMPLETED SUCCESSFULLY!\n')

    yield src

In [15]:
read_html_yield_names(variable()[3][1]['name'], variable()[4][1], 1)

<generator object read_html_yield_names at 0x00000251D17FB940>

# ОСНОВНОЙ класс "Article"
### for TWO objects - WORKING OPTION (для ДВУХ объектов - РАБОЧИЙ ВАРИАНТ)

In [81]:
# STARTING OPTION OPTION 4 = CLASS 18
class Article:
    html = '''
    <div class='findmeee'> blabla 
    <p> p-tag content
    <a href="https://chel/...">Как сделать ...?</a>
    <a href="https://svoy/...">Почему выгодно ...?</a>
    </p>
    </div>
    '''
    patt = ['div', 'findmeee']
    path_csv = variable()[4][0]
    host_1 = variable()[1][1]
    
    def __init__(self, src=html, pattern=patt, host=host_1, path=path_csv):
        self.src = src
        self.pattern = pattern
        self.path_csv = path
        self.host = host
                
# ---------------------------CHEL------------------------------------------        
class Vc(Article):
    def __init__(self, name='vc', src=Article.html, pattern=Article.patt, host=Article.host_1, path_csv=Article.path_csv):
        self.name = name
        self.src = src
        self.pattern = pattern
        self.host = host
        self.path_csv = path_csv
        
        self.soup = BeautifulSoup(self.src, 'lxml')
        self.blocks = self.soup.find(self.pattern[0], class_=self.pattern[1]).find_all('a')

    def __str__(self):
        return f"Chel:\nBlocks - {self.blocks}"

    def searching(self):
        self.articles_list = []
        self.count_link=1
        
        for item in self.blocks:
            self.title = item.text
            self.link = item.get("href")
            self.date_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print(f'Link №{self.count_link}. {self.title} {self.date_time}')

            self.articles_list.append(
                {
                    "Number": self.count_link,
                    "Topic": self.title,
                    "Link": self.link,
                    "DateTime": self.date_time
                }
            )

# ФУНКЦИЯ 5.0. !!! - Заполняем строки CSV-файла            
            save_csv(self.name, self.count_link, self.title, self.link, self.date_time, self.path_csv)
            self.count_link += 1
            sleep(random.randrange(1, 3)) # Для симуляции работы реального пользователя, а не робота
        return self.articles_list

# -----------------------------SVOY----------------------------------------
class Svoy(Article):
    def __init__(self, name='chelyab', src=Article.html, pattern=Article.patt, host=Article.host_1, path_csv=Article.path_csv):
        self.name = name
        self.src = src
        self.pattern = pattern
        self.host = host
        self.path_csv = path_csv
        
        self.soup = BeautifulSoup(self.src, 'lxml')
        self.blocks = self.soup.find_all(self.pattern[0], class_=self.pattern[1])

    def __str__(self):
        return f"Chel:\nBlocks - {self.blocks}"

    def searching(self):
        self.articles_list = []
        self.count_link=1
        
        for item in self.blocks:
            self.title = item.text.strip()
            self.link = self.host + item.find('a').get("href")
            self.date_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print(f'Link №{self.count_link}. {self.title} {self.date_time}')

            self.articles_list.append(
                {
                    "Number": self.count_link,
                    "Topic": self.title,
                    "Link": self.link,
                    "DateTime": self.date_time
                }
            )

# ФУНКЦИЯ 5.3. !!! - Заполняем строки CSV-файла            
            save_csv(self.name, self.count_link, self.title, self.link, self.date_time, self.path_csv)
            self.count_link += 1
            sleep(random.randrange(1, 3)) # Для симуляции работы реального пользователя, а не робота
        return self.articles_list

In [82]:
# Variant 4 - CHELYABINSK
# my_name = 'chelyabinsk'
# my_src = read_html(variable()[3][0]['name'], variable()[4][1])

my_name = variable()[3][0]['name']
my_src = read_html(my_name, variable()[4][0])
my_pattern = variable()[5][0]

# obj_chel = Chel(my_name, my_src, my_pattern)
obj_chel = Chel()
print('--------')
obj_chel.searching()

Fun.3.0. START of function "READ_HTML()"
1. Function "READ_HTML(CHELYABINSK)" is EXECUTING...
	In the folder "../data/processed/csv/" the file "barrels_articles_chelyabinsk" EXISTS.
	HTML-code "barrels_articles_chelyabinsk" is OPENED from ../data/processed/csv/barrels_articles_chelyabinsk.csv
Fun.3.0. "READ_HTML(CHELYABINSK)" COMPLETED SUCCESSFULLY!
--------
Link №1. Как сделать ...? 2025-02-17 20:10:30
Link №2. Почему выгодно ...? 2025-02-17 20:10:32


[{'Number': 1,
  'Topic': 'Как сделать ...?',
  'Link': 'https://chel/...',
  'DateTime': '2025-02-17 20:10:30'},
 {'Number': 2,
  'Topic': 'Почему выгодно ...?',
  'Link': 'https://svoy/...',
  'DateTime': '2025-02-17 20:10:32'}]

In [17]:
# 
print('-------------ARTICLES_LIST-------------')
obj_chel.articles_list

-------------ARTICLES_LIST-------------


[{'Number': 1,
  'Topic': 'Как сделать фундамент для бани-бочки?',
  'Link': 'https://chelyabinsk.bani.ru.com/poleznye-statiy/kak-sdelat-fundament-dlya-bani-bochki/',
  'DateTime': '2025-02-16 07:38:31'},
 {'Number': 2,
  'Topic': 'Печь в бане-бочке: печь с топкой через перегородку или на улицу — плюсы и минусы',
  'Link': 'https://chelyabinsk.bani.ru.com/poleznye-statiy/pech-v-bane-bochke-varianty-raspolozheniya/',
  'DateTime': '2025-02-16 07:38:32'},
 {'Number': 3,
  'Topic': 'Правила эксплуатации бань-бочек',
  'Link': 'https://chelyabinsk.bani.ru.com/poleznye-statiy/pravila-ekspluatacii-bani-bochki-kak-prodlit-srok-sluzhby/',
  'DateTime': '2025-02-16 07:38:32'},
 {'Number': 4,
  'Topic': 'Что нужно знать при покупке?',
  'Link': 'https://chelyabinsk.bani.ru.com/poleznye-statiy/chto-nuzhno-znat-pri-pokupke-bani-bochki/',
  'DateTime': '2025-02-16 07:38:32'},
 {'Number': 5,
  'Topic': 'Эксплуатация бани-бочки зимой',
  'Link': 'https://chelyabinsk.bani.ru.com/poleznye-statiy/eksplu

In [17]:
# Variant 4 - SVOY
# my_name = 'svoy'
# my_src = read_html(variable()[3][1]['name'], variable()[4][1])

my_name = variable()[3][1]['name']
my_src = read_html(my_name, variable()[4][1])
my_pattern = variable()[5][1]
obj_svoy = Svoy(my_name, my_src, my_pattern)

print('--------')
obj_svoy.searching()

Fun3. START of function "READ_HTML()"
1. Function "READ_HTML(SVOY-BANY)" is EXECUTING...
	In the folder "../data/interim/html/" the file "barrels_articles_svoy-bany" EXISTS.
	HTML-code "barrels_articles_svoy-bany" is OPENED from ../data/interim/html/barrels_articles_svoy-bany.html
Fun3. "READ_HTML(SVOY-BANY)" COMPLETED SUCCESSFULLY!
--------
Link №1. Технология строительства из минибруса 2025-02-16 16:25:28
Link №2. A-frame House: треугольные дома-шалаши снова набирают популярность 2025-02-16 16:25:28
Link №3. Баня из бруса или каркасная – что лучше выбрать? 2025-02-16 16:25:28
Link №4. Модульные бани: особенности строительства, преимущества конструкции 2025-02-16 16:25:28
Link №5. Плюсы и минусы каркасных домов 2025-02-16 16:25:28
Link №6. Правила эксплуатации бань-бочек 2025-02-16 16:25:28
Link №7. Напитки в бане: травяной чай 2025-02-16 16:25:28
Link №8. Как правильно париться в бане? 2025-02-16 16:25:28
Link №9. Как правильно запаривать банный веник из разных пород? 2025-02-16 16:2

[{'Number': 1,
  'Topic': 'Технология строительства из минибруса',
  'Link': '/article/tekhnologiya-stroitelstva-iz-minibrusa/',
  'DateTime': '2025-02-16 16:25:28'},
 {'Number': 2,
  'Topic': 'A-frame House: треугольные дома-шалаши снова набирают популярность',
  'Link': '/article/a-frame-house-treugolnye-doma-shalashi-snova-nabirayut-populyarnost/',
  'DateTime': '2025-02-16 16:25:28'},
 {'Number': 3,
  'Topic': 'Баня из бруса или каркасная – что лучше выбрать?',
  'Link': '/article/banya-iz-brusa-ili-karkasnaya-chto-luchshe-vybrat/',
  'DateTime': '2025-02-16 16:25:28'},
 {'Number': 4,
  'Topic': 'Модульные бани: особенности строительства, преимущества конструкции',
  'Link': '/article/vse-o-modulnyh-banyah/',
  'DateTime': '2025-02-16 16:25:28'},
 {'Number': 5,
  'Topic': 'Плюсы и минусы каркасных домов',
  'Link': '/article/plusy-i-minusy-karkasnyh-domov/',
  'DateTime': '2025-02-16 16:25:28'},
 {'Number': 6,
  'Topic': 'Правила эксплуатации бань-бочек',
  'Link': '/article/pravil

# ФУНКЦИЯ № 11. Основная функция для парсинга СКОПИРОВАННОГО HTML кода
### ФУНКЦИИ... Собираем все рабочие функции в одну главную с использованием ООП

In [80]:
# Каждый раз функция проходит по внутренней ссылке на товар и выполнение цикла приостанавливается благодаря команде !!! "YIELD" !!!
def definition_blocks_class():
# Определяем переменные с помощью функции VARIABLE()
    names = variable()[3]
    patt = variable()[5]
    
    paths = variable()[4]
    path_csv = paths[0]
    path_html = paths[1]
    path_json = paths[3]
    path_xlsx = paths[5]
    
    print(f'\t\tFunction 11.0 START of function "DEFINITION_BLOCKS()"\n')

    # ФУНКЦИЯ 5.0. !!! - Проверка наличия папки для файлов
    checking_folder(path_csv)
    
    # Счётчик для ссылок на САЙТЫ
    count = 1
    
    for item in range(len(names)):
        # Опредедяем на каком сайте мы находимся
        name = names[item]['name']
        # ФУНКЦИЯ 10.0. !!! - Чтение HTML файла для получения кода
        for src in read_html_yield_names(name, path_html, count):
        
            # Объявляем начало чтения ссылок
            print(f'\t{item+777}. START of function "DEFINITION_BLOCKS({name.upper()})" and it`s EXECUTING ...')

            # ФУНКЦИЯ 6.0. !!! - Заполняем заголовки для нового CSV-файла при его отсутствии
            prep_csv(name, path_csv)

            # Определяем паттерн для поиска блоков с информацией
            pattern = patt[item]
# -------------------------------------------------------------------------
            if name == 'chelyabinsk':
                obj = Chel(name, src, pattern)
# -------------------------------------------------------------------------
            elif name == 'svoy-bany':
                obj = Svoy(name, src, pattern)
# -------------------------------------------------------------------------    

            # Формируем список со статьями и ссылками, формируем CSV-файл
            obj.searching()
            
            # ФУНКЦИЯ 9.0. !!! - Заполняем JSON файл
            save_json(name, path_json, obj.articles_list)
        
        count += 1

        print(f'{item+777}. "DEFINITION_..({name.upper()})" COMPLETED SUCCESSFULLY!\n') # ======>
    print(f'\t\tFunction 11.0 END of function "DEFINITION_BLOCKS()"')
    # return name

# Безусловный запуск функции
if __name__ == '__main__':
    definition_blocks_class()

		Function 11.0 START of function "DEFINITION_BLOCKS()"

Fun.5.0. Folder "CSV" FOUND!!! It`s OK

77. Fun.10.0. START of function "READ_HTML_YIELD_NAMES()"
	Function "READ_HTML_YIELD_NAMES(CHELYABINSK)" is EXECUTING...
	In the folder "HTML" the file "barrels_articles_chelyabinsk" EXISTS.
	"HTML"-code "barrels_articles_chelyabinsk" is OPENED from "../data/interim/html/barrels_articles_chelyabinsk.html"
77. Fun.10.0. "READ_HTML_YIELD_NAMES(CHELYABINSK)" COMPLETED SUCCESSFULLY!

	777. START of function "DEFINITION_BLOCKS(CHELYABINSK)" and it`s EXECUTING ...
	Fun.6.1. START "PREP_CSV(CHELYABINSK)"
		The file "barrels_articles_chelyabinsk" already EXISTS in folder "CSV".
	Fun.6.1. END "PREP_CSV(CHELYABINSK)"
Link №1. Как сделать фундамент для бани-бочки? 2025-02-17 20:03:30
Link №2. Печь в бане-бочке: печь с топкой через перегородку или на улицу — плюсы и минусы 2025-02-17 20:03:31
Link №3. Правила эксплуатации бань-бочек 2025-02-17 20:03:31
Link №4. Что нужно знать при покупке? 2025-02-17 2

# PANDAS. Обрабатываем полученные данные

In [3]:
# Импортируем нужные нам библиотеки.
import pandas as pd
from io import StringIO
import numpy as np

In [ ]:
# file_shops = 'C:/Users/Андрей/Downloads/shops.csv'
barr = pd.DataFrame(columns=['Number', 'Topic', 'Link', 'DateTime'])
names = variable()[3]
path_csv = variable()[4][0]
path_xls = variable()[4][5]
start = 'barrels_articles_'
final = 'barrels_articles'
file_xls = os.path.join(path_xls['path_file'], final) + path_xls['extension_file']
writer = pd.ExcelWriter(file_xls, engine='xlsxwriter')

In [85]:
# Тренировочный
"""
writer = pd.ExcelWriter(file_xls, engine='xlsxwriter')
for item in range(len(names)):
    name = variable()[3][item]['name']
    file_start = start + name
    file_csv = os.path.join(path_csv['path_file'], file_start) + path_csv['extension_file']
    
    temp = pd.read_csv(file_csv)
    
    # Отправляем датафрейм в EXCEL-таблицу
    temp.to_excel(writer, sheet_name=name)
    
    barr = pd.concat([barr, temp])
writer.close()
"""

(148, 4)

In [10]:
# Проверка сколько записей на самом деле в CSV-файлах
path_csv = variable()[4][0]
names = variable()[3]
start = 'barrels_articles_'

for item in range(len(names)):
    name = variable()[3][item]['name']
    file_start = start + name
    file_csv = os.path.join(path_csv['path_file'], file_start) + path_csv['extension_file']

    temp = pd.read_csv(file_csv)
    print(file_csv)
    print(temp.shape)

../data/processed/csv/barrels_articles_chelyabinsk.csv
(112, 4)
../data/processed/csv/barrels_articles_svoy-bany.csv
(36, 4)


In [ ]:
barr.shape

In [87]:
# Сортируем с целью выявления дубликатов статей
barr.sort_values(['Number', 'DateTime'], ascending=[True, True]).head()

,Number,Topic,Link,DateTime
0,1,Как сделать фундамент для бани-бочки?,https://chelyabinsk.bani.ru.com/poleznye-stati...,2025-02-17 14:09:43
0,1,Технология строительства из минибруса,https://svoy-bany.ru/article/tekhnologiya-stro...,2025-02-17 14:10:27
28,1,Как сделать фундамент для бани-бочки?,https://chelyabinsk.bani.ru.com/poleznye-stati...,2025-02-17 14:13:19
9,1,Технология строительства из минибруса,https://svoy-bany.ru/article/tekhnologiya-stro...,2025-02-17 14:13:19
56,1,Как сделать фундамент для бани-бочки?,https://chelyabinsk.bani.ru.com/poleznye-stati...,2025-02-17 20:03:30


In [89]:
# Удаляем дубликаты строк с более поздней датой

# barr.duplicated().sum()
# barr.sort_values('Topic', ascending=False)
# barr.groupby(['Topic']).agg({'Number': 'count'})
# barr.groupby(['Topic'], as_index=False).agg({'Number': 'count'})
# barr.sort_values(['Number', 'DateTime'], ascending=[True, True]).groupby('Number').head(2)
barr.sort_values(['Number', 'DateTime'], ascending=[True, True]).groupby('Link').head(1)

,Number,Topic,Link,DateTime
0,1,Как сделать фундамент для бани-бочки?,https://chelyabinsk.bani.ru.com/poleznye-stati...,2025-02-17 14:09:43
0,1,Технология строительства из минибруса,https://svoy-bany.ru/article/tekhnologiya-stro...,2025-02-17 14:10:27
1,2,Печь в бане-бочке: печь с топкой через перегор...,https://chelyabinsk.bani.ru.com/poleznye-stati...,2025-02-17 14:09:45
1,2,A-frame House: треугольные дома-шалаши снова н...,https://svoy-bany.ru/article/a-frame-house-tre...,2025-02-17 14:10:29
2,3,Правила эксплуатации бань-бочек,https://chelyabinsk.bani.ru.com/poleznye-stati...,2025-02-17 14:09:46
2,3,Баня из бруса или каркасная – что лучше выбрать?,https://svoy-bany.ru/article/banya-iz-brusa-il...,2025-02-17 14:10:31
3,4,Что нужно знать при покупке?,https://chelyabinsk.bani.ru.com/poleznye-stati...,2025-02-17 14:09:47
3,4,"Модульные бани: особенности строительства, пре...",https://svoy-bany.ru/article/vse-o-modulnyh-ba...,2025-02-17 14:10:32
4,5,Эксплуатация бани-бочки зимой,https://chelyabinsk.bani.ru.com/poleznye-stati...,2025-02-17 14:09:48
4,5,Плюсы и минусы каркасных домов,https://svoy-bany.ru/article/plusy-i-minusy-ka...,2025-02-17 14:10:33


In [ ]:
# df = df.loc[df['STP'] != 1005092]
# barr.loc[barr[]]

# Функция № 21.0. предобработки данных из CSV файла

In [4]:
# Функция предобработки данных из CSV файла
def data_preprocessing():
    # Константы, пустой датафрейм для заполнения
    barr = pd.DataFrame(columns=['Number', 'Topic', 'Link', 'DateTime'])
    names = variable()[3]
    path_csv = variable()[4][0]
    path_xls = variable()[4][5]
    start = 'barrels_articles_'
    final = 'barrels_articles'
    file_xls = os.path.join(path_xls['path_file'], final) + path_xls['extension_file']
    
    # Конструктор для добавления информации в Excel
    writer = pd.ExcelWriter(file_xls, engine='xlsxwriter')
    
    for item in range(len(names)):
        # Находим в папке нужные CSV-файлы
        name = names[item]['name']
        file_start = start + name
        file_csv = os.path.join(path_csv['path_file'], file_start) + path_csv['extension_file']
        
        # Считываем CSV-данные во временный PD-файл
        temp = pd.read_csv(file_csv)
        # Удаляем из данных дубликаты с более поздними датами
        temp = temp.sort_values(['Number', 'DateTime'], ascending=[True, True]).groupby('Link').head(1)
        # Отправляем датафрейм в EXCEL-таблицу - с каждого сайта на разные страницы
        temp.to_excel(writer, sheet_name=name)
        # Сохраняем информацию из разных сайтов в один датафрейм
        barr = pd.concat([barr, temp])
    # Закрываем конструктор
    writer.close()
    return barr

In [5]:
df = data_preprocessing()
df.head(8)

,Number,Topic,Link,DateTime
0,1,Как сделать фундамент для бани-бочки?,https://chelyabinsk.bani.ru.com/poleznye-stati...,2025-02-17 14:09:43
1,2,Печь в бане-бочке: печь с топкой через перегор...,https://chelyabinsk.bani.ru.com/poleznye-stati...,2025-02-17 14:09:45
2,3,Правила эксплуатации бань-бочек,https://chelyabinsk.bani.ru.com/poleznye-stati...,2025-02-17 14:09:46
3,4,Что нужно знать при покупке?,https://chelyabinsk.bani.ru.com/poleznye-stati...,2025-02-17 14:09:47
4,5,Эксплуатация бани-бочки зимой,https://chelyabinsk.bani.ru.com/poleznye-stati...,2025-02-17 14:09:48
5,6,Почему покупатели выбирают Бани-бочки?,https://chelyabinsk.bani.ru.com/poleznye-stati...,2025-02-17 14:09:49
6,7,Планировки и внутреннее устройство бань-бочек,https://chelyabinsk.bani.ru.com/poleznye-stati...,2025-02-17 14:09:50
7,8,Выбор материала при производстве и отделке бан...,https://chelyabinsk.bani.ru.com/poleznye-stati...,2025-02-17 14:09:52


In [6]:
df.tail(10)

,Number,Topic,Link,DateTime
27,28,Секреты создания легкого и приятного пара,https://chelyabinsk.bani.ru.com/kak-sozdat-leg...,2025-02-17 14:10:26
0,1,Технология строительства из минибруса,https://svoy-bany.ru/article/tekhnologiya-stro...,2025-02-17 14:10:27
1,2,A-frame House: треугольные дома-шалаши снова н...,https://svoy-bany.ru/article/a-frame-house-tre...,2025-02-17 14:10:29
2,3,Баня из бруса или каркасная – что лучше выбрать?,https://svoy-bany.ru/article/banya-iz-brusa-il...,2025-02-17 14:10:31
3,4,"Модульные бани: особенности строительства, пре...",https://svoy-bany.ru/article/vse-o-modulnyh-ba...,2025-02-17 14:10:32
4,5,Плюсы и минусы каркасных домов,https://svoy-bany.ru/article/plusy-i-minusy-ka...,2025-02-17 14:10:33
5,6,Правила эксплуатации бань-бочек,https://svoy-bany.ru/article/pravila-ekspluata...,2025-02-17 14:10:35
6,7,Напитки в бане: травяной чай,https://svoy-bany.ru/article/napitki-v-bane-tr...,2025-02-17 14:10:37
7,8,Как правильно париться в бане?,https://svoy-bany.ru/article/kak-pravilno-pari...,2025-02-17 14:10:39
8,9,Как правильно запаривать банный веник из разны...,https://svoy-bany.ru/article/kak-pravilno-zapa...,2025-02-17 14:10:40


In [7]:
df.shape

(37, 4)

# ФУНКЦИИ № 19 и 20. Чтение HTML-кода, БЕЗ сохранения его в список. Команда YIELD. ПРЯМО из ИНТЕРНЕТА

In [83]:
# Функция для чтения HTML кода из Интернета на-лету
def read_html_yield(headers, url, name, count):
    # name = url.split('//')[1].split('.')[0]
    
    print(f'{count+32}. Fun.19.0. START of function "READ_HTML_YIELD({name.upper()})"')

    r = requests.get(url=url, headers=headers)
    src = r.text

    print(f'{count+32}. Fun.19.0. "READ_HTML_YIELD({name.upper()})" COMPLETED SUCCESSFULLY!\n')
    
    yield src

In [84]:
# INTERNET !!! "YIELD" !!!
def parsing_barrels_class():
# Определяем переменные с помощью функции VARIABLE()
    headers = variable()[0]
    # hosts = variable()[1]
    urls = variable()[2]
    names = variable()[3]
    patt = variable()[5]
    
    paths = variable()[4]
    path_csv = paths[0]
    path_html = paths[1]
    path_json = paths[3]
    
    print(f'\t\tFunction 20.0. START of function "PARSING_BARRELS_CLASS()"\n')

    # ФУНКЦИЯ 5.0. !!! - Проверка наличия папки для файлов
    checking_folder(path_csv)

    # Счётчик для ссылок на САЙТЫ
    count = 1
    
    for item in range(len(names)):
        # Опредедяем на каком сайте мы находимся
        name = names[item]['name']
        # ФУНКЦИЯ 19.0. !!! - Получение HTML кода из Интернета
        # for src in read_html_yield_names(names[item]['name'], path_html, count):
        for src in read_html_yield(headers, urls[item], name, count):
            
            # Объявляем начало чтения ссылок
            print(f'\t{item+777}. START of function "PARSING_BARRELS_CLASS({name.upper()})" and it`s EXECUTING ...')

            # ФУНКЦИЯ 6.1. !!! - Заполняем заголовки для нового CSV-файла при его отсутствии
            prep_csv(name, path_csv)

            # Определяем паттерн для поиска блоков с информацией
            pattern = patt[item]
# -------------------------------------------------------------------------
            if name == 'chelyabinsk':
                obj = Chel(name, src, pattern)
# -------------------------------------------------------------------------
            elif name == 'svoy-bany':
                obj = Svoy(name, src, pattern)
# -------------------------------------------------------------------------    

            # Формируем список со статьями и ссылками, формируем CSV-файл
            obj.searching()
            
            # ФУНКЦИЯ 9.0. !!! - Заполняем JSON файл
            save_json(name, path_json, obj.articles_list)
        
        count += 1
        
        print(f'{item+777}. "PARSING_..({name.upper()})" COMPLETED SUCCESSFULLY!\n')
    print(f'\t\tFunction 20.0. END of function "PARSING_BARRELS_CLASS()"')
    # return name

# Безусловный запуск функции
if __name__ == '__main__':
    parsing_barrels_class()

		Function 20.0. START of function "PARSING_BARRELS_CLASS()"

Fun.5.0. Folder "CSV" FOUND!!! It`s OK

33. Fun.19.0. START of function "READ_HTML_YIELD(CHELYABINSK)"
33. Fun.19.0. "READ_HTML_YIELD(CHELYABINSK)" COMPLETED SUCCESSFULLY!

	777. START of function "PARSING_BARRELS_CLASS(CHELYABINSK)" and it`s EXECUTING ...
	Fun.6.1. START "PREP_CSV(CHELYABINSK)"
		The file "barrels_articles_chelyabinsk" already EXISTS in folder "CSV".
	Fun.6.1. END "PREP_CSV(CHELYABINSK)"
Link №1. Как сделать фундамент для бани-бочки? 2025-02-17 20:14:59
Link №2. Печь в бане-бочке: печь с топкой через перегородку или на улицу — плюсы и минусы 2025-02-17 20:15:01
Link №3. Правила эксплуатации бань-бочек 2025-02-17 20:15:04
Link №4. Что нужно знать при покупке? 2025-02-17 20:15:05
Link №5. Эксплуатация бани-бочки зимой 2025-02-17 20:15:06
Link №6. Почему покупатели выбирают Бани-бочки? 2025-02-17 20:15:07
Link №7. Планировки и внутреннее устройство бань-бочек 2025-02-17 20:15:09
Link №8. Выбор материала при пр

# FORUM